# Affine registration

This notebook presents how series of source images can be fixed by detecting affine transformations between them and applying the shift.

It is usefull in terms of model fitting to skip the affine transformation part and focus on pure fitting.

It is capable of processing multiple images in parallel.

## Data
Data for the neuroscience use case is available at 
[HCP](https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS)

Following code assumes that you have a file '.aws/credentials', 
that includes a section with credentials needed to access HCP data.
```
[hcp]
AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXX
```

### Browsing the data on AWS S3
Single case data consists of 3 files `bvals`, `bvecs` and `data.nii.gz`. Following command lists available cases:
```
aws s3 ls s3://hcp-openaccess/HCP/ --profile hcp
```



In [4]:
case_id = 994273

import AFQ.data as afd


data_dict = afd.fetch_hcp([case_id])

In [5]:
import os.path as op
home = op.expanduser('~')

import nibabel as nib
img = nib.load(op.join(home, 'AFQ_data/HCP/sub-{0}/sess-01/dwi/sub-{0}_dwi.nii.gz'.format(case_id)))
data = img.get_data()

In [9]:
from queue import Queue, Empty
from threading import Thread, Lock

class FixerThread(Thread):
    def __init__(self, thread_id, task_queue, lock, data, fixed_data):
        Thread.__init__(self)
        self.data = data
        self.thread_id = thread_id
        self.task_queue = task_queue
        self.lock = lock

    def run(self):
        try:
            while True:
                idx = self.task_queue.get_nowait()
                fixed_image, _ = afr.affine_registration(data[..., idx], data[..., 0])
                self.lock.acquire()
                fixed_data[..., idx] = fixed_image
                self.lock.release()
        except Empty:
            pass

In [ ]:
import numpy as np
import datetime

fixed_data = np.array(data, copy=True)
task_queue = Queue()
lock = Lock()

for idx in range(1, 288):
    task_queue.put(idx)
    
threads = [FixerThread(thread_id, task_queue, lock, data, fixed_data) for thread_id in range(12)]
for thread in threads:
    print('{0} starting thread'.format(datetime.datetime.now()))
    thread.start()
    
for thread in threads:
    print('{0} joining thread'.format(datetime.datetime.now()))
    thread.join()
    
print('{0} all threads joined'.format(datetime.datetime.now()))
nib.save(fixed_data, os.path.join(home,'fixed-{0}.nii.gz'.format(case_id)))
